In [1]:
from itertools import product
from copy import deepcopy

In [2]:
def isLinearlyIndependent(S):
    M = matrix([list(t) for t in S])
    if len(S) < n+1:
        if M.rank() == len(S):
            return True
        return False
    else:
        if any(l == 0 for l in M.minors(n+1)):
            return False
        return True

In [3]:
def calculate_periodic_points(f, g):
    return Set(f.periodic_points(1)), Set(g.periodic_points(1))

In [4]:
def initialize_dictionaries(f, g, L, K, mult_to_point_L, mult_to_point_K, point_to_mult_L, point_to_mult_K, repeated_mult_L):
    for i in range(len(L)):
        mult_L = f.multiplier(L[i], 1).charpoly()
        mult_K = g.multiplier(K[i], 1).charpoly()
        tup_L = (mult_L, level)
        tup_K = (mult_K, level)
        if tup_L not in mult_to_point_L:
            mult_to_point_L[tup_L] = [L[i]]
        else:
            mult_to_point_L[tup_L] += [L[i]]
        if tup_K not in mult_to_point_K:
            mult_to_point_K[tup_K] = [K[i]]
        else:
            mult_to_point_K[tup_K] += [K[i]]
        point_to_mult_L[L[i]] = (mult_L, 0)
        point_to_mult_K[K[i]] = (mult_K, 0)
    for mult_L in mult_to_point_L:
        repeated = len(mult_to_point_L[mult_L])
        if mult_L not in mult_to_point_K:
            pass
        elif len(mult_to_point_K[mult_L]) != repeated:
            pass
        if repeated not in repeated_mult_L:
            repeated_mult_L[repeated] = [mult_to_point_L[mult_L]]
        else:
            repeated_mult_L[repeated] += [mult_to_point_L[mult_L]]

In [5]:
def find_independent_set(repeated_mult_L, source, corresponding):
    more = True
    for r in sorted(repeated_mult_L.keys()):
        for point_lst in repeated_mult_L[r]:
            for point in point_lst:
                if isLinearlyIndependent(source + [point]):
                    source.append(point)
                    mult = point_to_mult_L[point]
                    # if another point with this multiplier and level pair is in S
                    # then the multiplier level pair will be the last element of corresponding
                    if len(corresponding) != 0:
                        if corresponding[-1][0] == mult:
                            corresponding[-1][1] += 1
                        else:
                            corresponding.append([mult, 1])
                    else:
                        corresponding.append([mult, 1])
                if len(source) == n+2:
                    more = False
                    break
            if len(source) == n+2:
                break
        if len(source) == n+2:
            break
    return more

In [6]:
def find_preimages(i_repeated_mult, point_to_mult_L, mult_to_point_K, f, g, mult_to_point_L, point_to_mult_K, repeated_mult_L, a_repeated_mult):
    for r in i_repeated_mult:
            for point_lst_L in i_repeated_mult[r]:
                old_tup_L = point_to_mult_L[point_lst_L[0]]
                #print('current tup:', old_tup_L)
                point_lst_K = mult_to_point_K[old_tup_L]
                mult_L = old_tup_L[0]
                Tl = []
                Tk = []
                for pnt in point_lst_L:
                    for preimage in f.rational_preimages(pnt):
                        if preimage != pnt:
                            Tl.append(preimage)
                for pnt in point_lst_K:
                    for preimage in g.rational_preimages(pnt):
                        if preimage != pnt:
                            Tk.append(preimage)
                #print('Tl:', Tl)
                #print('Tk:', Tk)
                if len(Tl) != len(Tk):
                    return []
                if len(Tl) != 0:
                    found_no_more = False
                    new_tup_L = (mult_L, level)
                    new_tup_K = (mult_L, level)
                    #print('new_tup_L:', new_tup_L)
                    mult_to_point_L[new_tup_L] = Tl
                    mult_to_point_K[new_tup_K] = Tk
                    for i in range(len(Tl)):
                        point_to_mult_L[Tl[i]] = new_tup_L
                        point_to_mult_K[Tk[i]] = new_tup_K
                    repeated = len(Tl)
                    if repeated not in repeated_mult_L:
                        repeated_mult_L[repeated] = [Tl]
                    else:
                        repeated_mult_L[repeated] += [Tl]
                    if repeated not in a_repeated_mult:
                        a_repeated_mult[repeated] = [Tl]
                    else:
                        a_repeated_mult[repeated] += [Tl]
                    # right now rechecking independence
                    source = []
                    corresponding = []
                    more = find_independent_set(repeated_mult_L, source, corresponding)
                if not more:
                    break
            if not more:
                break
    return more, found_no_more, source, corresponding

In [7]:
def initialize_interator(corresponding, possible_targets, mult_to_point_K, arrangements_iterators):
    for tup in corresponding:
        possible_targets.append([mult_to_point_K[tup[0]], tup[1]])
    #print('possible_targets:', possible_targets)
    for lst in possible_targets:
        arrangements_iterators.append(Arrangements(range(len(lst[0])), lst[1]))

In [42]:
def find_conjugations(arrangements_iterators, possible_targets, source, Conj, f, g):
    for tup in product(*arrangements_iterators):
        current_target = []
        for i in range(len(tup)):
            for j in tup[i]:
                current_target.append(possible_targets[i][0][j])
        #print('current_target:', current_target)
        try:
            phi = f.domain().point_transformation_matrix(current_target, source)
            #print('phi:', phi)
            #print('phi inverse', phi.inverse())
            #print([phi*pnt for pnt in current_target])
            #%time f.conjugate(phi) == g
            if f.conjugate(phi) == g:
                Conj.append(phi)
        except ValueError:
            pass

In [9]:
K.<v>=CyclotomicField(3)
P.<x,y,z> = ProjectiveSpace(K, 2)
f = DynamicalSystem([8*x^7 - 35*x^4*y^3 - 35*x^4*z^3 - 7*x*y^6 - 140*x*y^3*z^3 - 7*x*z^6, -7*x^6*y - 35*x^3*y^4 - 140*x^3*y*z^3 + 8*y^7 - 35*y^4*z^3 - 7*y*z^6,-7*x^6*z - 140*x^3*y^3*z - 35*x^3*z^4 - 7*y^6*z - 35*y^3*z^4 + 8*z^7])
g = f

In [10]:
L, K = calculate_periodic_points(f,g)

In [11]:
%%timeit
L, K = calculate_periodic_points(f,g)

436 ms ± 5.72 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
mult_to_point_L = {}
mult_to_point_K = {}
point_to_mult_L = {}
point_to_mult_K = {}
repeated_mult_L = {}
level = 0
initialize_dictionaries(f, g, L, K, mult_to_point_L, mult_to_point_K, point_to_mult_L, point_to_mult_K, repeated_mult_L)

In [13]:
%%timeit
mult_to_point_L = {}
mult_to_point_K = {}
point_to_mult_L = {}
point_to_mult_K = {}
repeated_mult_L = {}
level = 0
initialize_dictionaries(f, g, L, K, mult_to_point_L, mult_to_point_K, point_to_mult_L, point_to_mult_K, repeated_mult_L)

667 ms ± 19.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
source = []
corresponding = []
n = f.domain().dimension_relative()
more = find_independent_set(repeated_mult_L, source, corresponding)
more

False

In [15]:
%%timeit
source = []
corresponding = []
n = f.domain().dimension_relative()
more = find_independent_set(repeated_mult_L, source, corresponding)

7.73 ms ± 220 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [16]:
i_repeated_mult = deepcopy(repeated_mult_L)
a_repeated_mult = {}
found_no_more = True
while more:
    level += 1
    more, found_no_more, source, corresponding = (i_repeated_mult, point_to_mult_L, mult_to_point_K, f, g, mult_to_point_L, point_to_mult_K, repeated_mult_L, a_repeated_mult)
    if found_no_more:
        raise ValueError('no more rational preimages')
    i_repeated_mult = deepcopy(a_repeated_mult)
    a_repeated_mult = {}
    found_no_more = True

In [13]:
possible_targets = []
arrangements_iterators = []
initialize_interator(corresponding, possible_targets, mult_to_point_K, arrangements_iterators)

In [18]:
%%timeit
possible_targets = []
arrangements_iterators = []
initialize_interator(corresponding, possible_targets, mult_to_point_K, arrangements_iterators)

7.8 µs ± 52 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [43]:
%%time
Conj = []
find_conjugations(arrangements_iterators, possible_targets, source, Conj, f, g)

CPU times: user 13.2 s, sys: 18.7 ms, total: 13.2 s
Wall time: 13.2 s


In [29]:
%%timeit
for tup in product(*arrangements_iterators):
    pass

4.96 s ± 19.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
arrangements_iterators[0].cardinality()

3024

In [76]:
#%%timeit
tup = t[6]
current_target = []
for i in range(len(tup)):
    for j in tup[i]:
        current_target.append(possible_targets[i][0][j])
#print('current_target:', current_target)
try:
    %time phi = f.domain().point_transformation_matrix(current_target, source)
    #print('phi:', phi)
    #print('phi inverse', phi.inverse())
    #print([phi*pnt for pnt in current_target])
    #k = f.conjugate(phi)
    #bo = k == g
    %time k = f.conjugate(phi)
    %time bo = k == g
    if bo:
        Conj.append(phi)
except ValueError:
    pass

CPU times: user 4.91 ms, sys: 0 ns, total: 4.91 ms
Wall time: 4.91 ms
CPU times: user 16.3 ms, sys: 0 ns, total: 16.3 ms
Wall time: 15.9 ms
CPU times: user 500 µs, sys: 0 ns, total: 500 µs
Wall time: 502 µs


In [48]:
t = list(product(*arrangements_iterators))

In [21]:
sys.getsizeof(t)

11746976

In [37]:
len(t)

1413720

In [43]:
sys.getsizeof(t[2])

48

In [46]:
size = get_size(t)

In [44]:
import sys

def get_size(obj, seen=None):
    """Recursively finds size of objects"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if isinstance(obj, dict):
        size += sum([get_size(v, seen) for v in obj.values()])
        size += sum([get_size(k, seen) for k in obj.keys()])
    elif hasattr(obj, '__dict__'):
        size += get_size(obj.__dict__, seen)
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum([get_size(i, seen) for i in obj])
    return size

In [49]:
RR(size*10**-6)

351.039776000000

In [56]:
Conj

[]

In [22]:
def test_conj(tup):
    current_target = []
    for i in range(len(tup)):
        for j in tup[i]:
            current_target.append(possible_targets[i][0][j])
    try:
        phi = f.domain().point_transformation_matrix(current_target, source)
        if f.conjugate(phi) == g:
            return phi
    except ValueError:
        pass
    return None

In [23]:
test_conj(t[0])

[1 0 0]
[0 1 0]
[0 0 1]

In [24]:
from sage.parallel.use_fork import p_iter_fork

In [49]:
parallel_data = []
for tup in t:
    parallel_data.append(([tup], {}))

"""X = p_iter_fork(2)

conj = []
for ret in X(test_conj, parallel_data):
    print(ret[0])
    if ret[1]:
        conj.append(ret[1])"""
print(parallel_data)

[([([0, 1, 2, 3],)], {}), ([([0, 1, 2, 4],)], {}), ([([0, 1, 2, 5],)], {}), ([([0, 1, 2, 6],)], {}), ([([0, 1, 2, 7],)], {}), ([([0, 1, 2, 8],)], {}), ([([0, 1, 3, 2],)], {}), ([([0, 1, 3, 4],)], {}), ([([0, 1, 3, 5],)], {}), ([([0, 1, 3, 6],)], {}), ([([0, 1, 3, 7],)], {}), ([([0, 1, 3, 8],)], {}), ([([0, 1, 4, 2],)], {}), ([([0, 1, 4, 3],)], {}), ([([0, 1, 4, 5],)], {}), ([([0, 1, 4, 6],)], {}), ([([0, 1, 4, 7],)], {}), ([([0, 1, 4, 8],)], {}), ([([0, 1, 5, 2],)], {}), ([([0, 1, 5, 3],)], {}), ([([0, 1, 5, 4],)], {}), ([([0, 1, 5, 6],)], {}), ([([0, 1, 5, 7],)], {}), ([([0, 1, 5, 8],)], {}), ([([0, 1, 6, 2],)], {}), ([([0, 1, 6, 3],)], {}), ([([0, 1, 6, 4],)], {}), ([([0, 1, 6, 5],)], {}), ([([0, 1, 6, 7],)], {}), ([([0, 1, 6, 8],)], {}), ([([0, 1, 7, 2],)], {}), ([([0, 1, 7, 3],)], {}), ([([0, 1, 7, 4],)], {}), ([([0, 1, 7, 5],)], {}), ([([0, 1, 7, 6],)], {}), ([([0, 1, 7, 8],)], {}), ([([0, 1, 8, 2],)], {}), ([([0, 1, 8, 3],)], {}), ([([0, 1, 8, 4],)], {}), ([([0, 1, 8, 5],)], {}),

In [29]:
len(parallel_data)

1413720

In [16]:
possible_targets = []
subset_iterators = []
for tup in corresponding:
    possible_targets.append([mult_to_point_K[tup[0]], tup[1]])
#print('possible_targets:', possible_targets)
for lst in possible_targets:
    subset_iterators.append(Subsets(range(len(lst[0])), lst[1]))

In [28]:
for i in Arrangements(list(subset_iterators[0][0]), len(list(subset_iterators[0][0]))):
    print(i)

[0, 1, 2, 3]
[0, 1, 3, 2]
[0, 2, 1, 3]
[0, 2, 3, 1]
[0, 3, 1, 2]
[0, 3, 2, 1]
[1, 0, 2, 3]
[1, 0, 3, 2]
[1, 2, 0, 3]
[1, 2, 3, 0]
[1, 3, 0, 2]
[1, 3, 2, 0]
[2, 0, 1, 3]
[2, 0, 3, 1]
[2, 1, 0, 3]
[2, 1, 3, 0]
[2, 3, 0, 1]
[2, 3, 1, 0]
[3, 0, 1, 2]
[3, 0, 2, 1]
[3, 1, 0, 2]
[3, 1, 2, 0]
[3, 2, 0, 1]
[3, 2, 1, 0]


In [29]:
for tup in product(*arrangements_iterators):
    print(tup)

([0, 1, 2, 3],)
([0, 1, 2, 4],)
([0, 1, 2, 5],)
([0, 1, 2, 6],)
([0, 1, 2, 7],)
([0, 1, 2, 8],)
([0, 1, 3, 2],)
([0, 1, 3, 4],)
([0, 1, 3, 5],)
([0, 1, 3, 6],)
([0, 1, 3, 7],)
([0, 1, 3, 8],)
([0, 1, 4, 2],)
([0, 1, 4, 3],)
([0, 1, 4, 5],)
([0, 1, 4, 6],)
([0, 1, 4, 7],)
([0, 1, 4, 8],)
([0, 1, 5, 2],)
([0, 1, 5, 3],)
([0, 1, 5, 4],)
([0, 1, 5, 6],)
([0, 1, 5, 7],)
([0, 1, 5, 8],)
([0, 1, 6, 2],)
([0, 1, 6, 3],)
([0, 1, 6, 4],)
([0, 1, 6, 5],)
([0, 1, 6, 7],)
([0, 1, 6, 8],)
([0, 1, 7, 2],)
([0, 1, 7, 3],)
([0, 1, 7, 4],)
([0, 1, 7, 5],)
([0, 1, 7, 6],)
([0, 1, 7, 8],)
([0, 1, 8, 2],)
([0, 1, 8, 3],)
([0, 1, 8, 4],)
([0, 1, 8, 5],)
([0, 1, 8, 6],)
([0, 1, 8, 7],)
([0, 2, 1, 3],)
([0, 2, 1, 4],)
([0, 2, 1, 5],)
([0, 2, 1, 6],)
([0, 2, 1, 7],)
([0, 2, 1, 8],)
([0, 2, 3, 1],)
([0, 2, 3, 4],)
([0, 2, 3, 5],)
([0, 2, 3, 6],)
([0, 2, 3, 7],)
([0, 2, 3, 8],)
([0, 2, 4, 1],)
([0, 2, 4, 3],)
([0, 2, 4, 5],)
([0, 2, 4, 6],)
([0, 2, 4, 7],)
([0, 2, 4, 8],)
([0, 2, 5, 1],)
([0, 2, 5, 3],)
([0, 2, 

In [40]:
%%time
Conj = []
count = 0
count2 = 0
for tup in product(*subset_iterators):
    target_set = []
    for i in range(len(tup)):
        for j in tup[i]:
            target_set.append(possible_targets[i][0][j])
    if isLinearlyIndependent(target_set):
        subset_arrangements = []
        for subset in tup:
            subset_arrangements.append(Arrangements(subset, len(subset)))
        for tup in product(*subset_arrangements):
            count2 += 1
            current_target = []
            for i in range(len(tup)):
                for j in tup[i]:
                    current_target.append(possible_targets[i][0][j])
            phi = f.domain().point_transformation_matrix(current_target, source)
            if f.conjugate(phi) == g:
                Conj.append(phi)
    else:
        count += 1
print('number not independent:', count)
print('number of inner loop runs:', count2)

number not independent: 72
number of inner loop runs: 1296
CPU times: user 11.5 s, sys: 49.4 ms, total: 11.5 s
Wall time: 11.5 s


In [46]:
def subset_test_conj(tuples):
    Conj = []
    for tup in tuples:
        target_set = []
        for i in range(len(tup)):
            for j in tup[i]:
                target_set.append(possible_targets[i][0][j])
        if isLinearlyIndependent(target_set):
            subset_arrangements = []
            for subset in tup:
                subset_arrangements.append(Arrangements(subset, len(subset)))
            for tup in product(*subset_arrangements):
                current_target = []
                for i in range(len(tup)):
                    for j in tup[i]:
                        current_target.append(possible_targets[i][0][j])
                phi = f.domain().point_transformation_matrix(current_target, source)
                if f.conjugate(phi) == g:
                    Conj.append(phi)
    return Conj

In [44]:
from sage.parallel.use_fork import p_iter_fork

In [54]:
%%time
all_subsets = list(product(*subset_iterators))
num_cpus = 4
parallel_data = []
for i in range(num_cpus):
    start = floor(len(all_subsets)*i/num_cpus)
    end = floor(len(all_subsets)*(i+1)/num_cpus)
    tuples = all_subsets[start:end]
    parallel_data.append(([tuples], {}))

X = p_iter_fork(num_cpus)

conj = []
for ret in X(subset_test_conj, parallel_data):
    if ret[1]:
        conj += ret[1]

CPU times: user 27.2 ms, sys: 24.5 ms, total: 51.7 ms
Wall time: 3.44 s


In [65]:
Set(Conj) == Set(conj)

False

In [58]:
l = [1,2,3]
l[1:3]

[2, 3]

In [59]:
len(l)

3

In [66]:
for i in range(216):
    print(Conj[i] in conj)

True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [63]:
print(Conj[1])

[     1      v      v]
[     1      1 -v - 1]
[     1 -v - 1      1]


In [64]:
print(conj[1])

[     0 -v - 1      0]
[     v      0      0]
[     0      0      1]
